In [ ]:
import pandas as pd
import json
import random

from tqdm import tqdm

#from transformer_lens import HookedTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

import re

import torch
from torch.utils.data import DataLoader
import gc